In [269]:
# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle

username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)
oracle_cur = connection.cursor()
sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER where VENDOR_NBR IN ( '8571' ,'445932', '447709')"

oracle_df = pd.read_sql(sql_query, connection)
oracle_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
print(oracle_df)

   VENDOR_SITE_ID VENDOR_NBR         VENDOR_NAME   VENDOR_START_DATE  \
0         3275707     445932                None 2022-07-20 09:38:30   
1         3276057     447709                None 2022-08-05 08:17:43   
2         3218150     008571  McELRONE SALES INC 2022-02-25 11:02:50   

  VENDOR_END_DATE VENDOR_STATUS   VENDOR_TYPE  AP_VENDOR_ID  \
0            None             A  Buyer Vendor       1935048   
1            None             A  Buyer Vendor       1935048   
2            None             A        Broker        992011   

                    AP_VENDOR_NAME AP_VENDOR_ALT_NAME  ... EOQ_CAP_DAYS  \
0  INTERNATIONAL TRADE IMPORTS LLC               None  ...            0   
1  INTERNATIONAL TRADE IMPORTS LLC               None  ...            0   
2              C&S Dummy Broker AP               None  ...         None   

  WHSE_SHIP_TO SHIP_WITH_VENDOR RANK_PERCENT_OVERRIDE PO_DATE_CHANGE_RSN_FLAG  \
0            0           000000                     0                    Non

In [28]:
oracle_df

,VENDOR_SITE_ID,VENDOR_NBR,VENDOR_NAME,VENDOR_START_DATE,VENDOR_END_DATE,VENDOR_STATUS,VENDOR_TYPE,AP_VENDOR_ID,AP_VENDOR_NAME,AP_VENDOR_ALT_NAME,...,PICKUP_DAY_6,PICKUP_DAY_7,EOQ_CAP_DAYS,WHSE_SHIP_TO,SHIP_WITH_VENDOR,RANK_PERCENT_OVERRIDE,PO_DATE_CHANGE_RSN_FLAG,PULLBKS_AVAILABLE,HAWAII_TRANS_METHOD,TEST_NEW
0,3275707,445932,INTL. TRADE IMPORTS,2022-07-20,None,A,Buyer Vendor,1935048,INTERNATIONAL TRADE IMPORTS LLC,None,...,None,None,0,0,000000,0,None,None,None,None
1,3276057,447709,INTNL TRADE IMPORTS LLC,2022-08-05,None,A,Buyer Vendor,1935048,INTERNATIONAL TRADE IMPORTS LLC,None,...,None,None,0,0,000000,0,None,None,None,None
2,3218150,008571,McELRONE SALES INC,2022-02-25,None,A,Broker,992011,C&S Dummy Broker AP,None,...,None,None,None,None,None,None,None,None,None,None


In [107]:
import numpy as np

oracle_df = oracle_df.applymap(lambda x: str(x) if x is not None else np.nan)

# Convert 'VENDOR_STATUS' to string type
oracle_df['BUYER_NAME'] = oracle_df['BUYER_NAME'].astype(str)

# Check the size of characters in 'VENDOR_STATUS' column
size_of_characters = oracle_df['BUYER_NAME'].str.len()

# Display the result
print(size_of_characters)
oracle_df['BUYER_NAME']

0    16
1    16
2     3
Name: BUYER_NAME, dtype: int64


0    Lougee, Brittany
1    Lougee, Brittany
2                 nan
Name: BUYER_NAME, dtype: object

In [50]:
# oracle_query = '''select *  from APPS.XXAP_VENDOR_MASTER where BROKER_VENDOR IN ( '8571' ,'445932', '447709')'''
oracle_df
oracle_df
# Assuming 'VENDOR_HOME_LOC_DEST' is the column in your DataFrame
# oracle_df['VENDOR_HOME_LOC_DEST'] = oracle_df['VENDOR_HOME_LOC_DEST'].astype(int)
oracle_df.loc[ oracle_df['VENDOR_TYPE'] == 'Broker', 'VENDOR_TYPE'] = 'B'
oracle_df.loc[oracle_df['VENDOR_TYPE'] == 'Buyer Vendor', 'VENDOR_TYPE'] = 'V'
oracle_df = oracle_df.fillna('')
upsert_vendor_master_and_childs_oracle(oracle_df,post_cur)


insert if
insert if
insert if


In [47]:
oracle_df['VENDOR_HOME_LOC_DEST'] = pd.to_numeric(oracle_df['VENDOR_HOME_LOC_DEST'], errors='coerce').fillna(0).astype(int)


In [ ]:
oracle_df

In [95]:
request_id = 1
current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
def upsert_vendor_master_and_childs_oracle(oracle_df,cur):
    # Iterate over DataFrame rows for upsert
    for index, row in oracle_df.iterrows():
        # cur.execute("SELECT nextval('cs_vendor_id_seq')")
        # next_sequence_value = cur.fetchone()[0]
        
        # print(next_sequence_value)
        cur.execute(
        f'''INSERT INTO cspoms.XXPO_VENDOR_MASTER (
            CS_VENDOR_ID,
            VENDOR_NUMBER,
            VENDOR_FACILITY,
            CS_VEND_SITE_ID,
            CONNECTING_VENDOR,
            VENDOR_NAME,
            VENDOR_TYPE,
            FACILITY_NAME,
            VENDOR_STATUS,
            AP_VENDOR_NBR,
            AP_VENDOR_NAME,
            BUYER_ID,
            BUYER_NAME,
            FOOD_LINK_FLAG,
            ADDR_LINE1,
            ADDR_LINE2,
            ADDR_LINE3,
            CITY,
            STATE,
            ZIP_CODE,
            COUNTRY,
            PHONE_NBR,
            VENDOR_CONTACT_NAME,
            SHIP_TO_LOCATION_NAME,
            BILL_TO_LOCATION_NAME,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        ) 
        VALUES (
            
             nextval('cs_vendor_id_seq'),
            '{row['VENDOR_NBR']}'::numeric,
            '{row['VENDOR_HOME_LOC_DEST']}'::numeric,
            '{row['VENDOR_SITE_ID']}',
            '{row['CONNECTING_VENDOR']}',
             %(vendor_name)s,
            '{row['VENDOR_TYPE']}',
            '{row['SHIP_TO_LOCATION_NAME']}',
            '{row['VENDOR_STATUS']}',
            '{row['AP_VENDOR_NBR']}',
             %(ap_vendor_name)s,
            '{row['BICEPS_BUYER_CODE']}',
            '{row['BUYER_NAME']}',
            '{row['VENDOR_FOODLINK_FLG']}',
            '{row['ADDR_LINE1']}',
            '{row['ADDR_LINE2']}',
            '{row['ADDR_LINE3']}',
            '{row['CITY']}',
            '{row['STATE']}',
            '{row['ZIP_CODE']}',
            '{row['COUNTRY']}',
            '{row['PHONE_NBR']}',
            '{row['VENDOR_CONTACT_NAME']}',
            '{row['SHIP_TO_LOCATION_NAME']}',
            '{row['BILL_TO_LOCATION_NAME']}',
            -1,
                TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
                TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
            '-1',
            '-1'
        ) 
        ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET 
            VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
            VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY,
            CS_VEND_SITE_ID = EXCLUDED.CS_VEND_SITE_ID,
            CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
            VENDOR_NAME = EXCLUDED.VENDOR_NAME,
            VENDOR_TYPE = EXCLUDED.VENDOR_TYPE,
            FACILITY_NAME = EXCLUDED.FACILITY_NAME,
            VENDOR_STATUS = EXCLUDED.VENDOR_STATUS,
            AP_VENDOR_NBR = EXCLUDED.AP_VENDOR_NBR,
            AP_VENDOR_NAME = EXCLUDED.AP_VENDOR_NAME,
            BUYER_ID = EXCLUDED.BUYER_ID,
            BUYER_NAME = EXCLUDED.BUYER_NAME,
            FOOD_LINK_FLAG = EXCLUDED.FOOD_LINK_FLAG,
            ADDR_LINE1 = EXCLUDED.ADDR_LINE1,
            ADDR_LINE2 = EXCLUDED.ADDR_LINE2,
            ADDR_LINE3 = EXCLUDED.ADDR_LINE3,
            CITY = EXCLUDED.CITY,
            STATE = EXCLUDED.STATE,
            ZIP_CODE = EXCLUDED.ZIP_CODE,
            COUNTRY = EXCLUDED.COUNTRY,
            PHONE_NBR = EXCLUDED.PHONE_NBR,
            VENDOR_CONTACT_NAME = EXCLUDED.VENDOR_CONTACT_NAME,
            SHIP_TO_LOCATION_NAME = EXCLUDED.SHIP_TO_LOCATION_NAME,
            BILL_TO_LOCATION_NAME = EXCLUDED.BILL_TO_LOCATION_NAME,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
            RETURNING CS_VENDOR_ID''',{'vendor_name': row['VENDOR_NAME'],'ap_vendor_name': row['AP_VENDOR_NAME']}
    )


        # Fetch the generated C1 value
        generated_c1 = cur.fetchone()[0]

        # If C1 was generated or updated, insert/update other tables
        if generated_c1:
            print('insert if')
            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_ORDER_POINT (
            CS_VENDOR_ID,
            MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO,
            USE_CYCLE_STOCK_AS_MIN,
            USE_FAC_WHSE_SS,
            LEAD_TIME_OVRD_DAYS_1,
            LEAD_TIME_OVRD_DAYS_2,
            LEAD_TIME_OVRD_DAYS_3,
            LEAD_TIME_OVRD_DAYS_4,
            LEAD_TIME_OVRD_DAYS_5,
            LEAD_TIME_OVRD_DAYS_6,
            LEAD_TIME_OVRD_DAYS_7,
            APPLY_RATIONING,
            OP_DAYS_1ST_RUN_1,
            OP_DAYS_1ST_RUN_2,
            OP_DAYS_1ST_RUN_3,
            OP_DAYS_1ST_RUN_4,
            OP_DAYS_1ST_RUN_5,
            OP_DAYS_1ST_RUN_6,
            OP_DAYS_1ST_RUN_7,
            OP_DAYS_2ND_RUN_1,
            OP_DAYS_2ND_RUN_2,
            OP_DAYS_2ND_RUN_3,
            OP_DAYS_2ND_RUN_4,
            OP_DAYS_2ND_RUN_5,
            OP_DAYS_2ND_RUN_6,
            OP_DAYS_2ND_RUN_7,
            SHIP_WITH_VENDOR,
            CONNECTING_VENDOR,
            HAWAII_TRANS_METHOD,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            '{row['MAIN_FACILITY']}',
            '{row['BUILD_TO_DAYS_SUPPLY']}',
            '{row['DAYS_TO_BUILD_TO']}',
            '{row['USE_CYCLE_STOCK_AS_MIN']}',
            '{row['USE_FAC_WHSE_SS']}',
            '{row['LEAD_TIME_OVRD_DAYS_1']}',
            '{row['LEAD_TIME_OVRD_DAYS_2']}',
            '{row['LEAD_TIME_OVRD_DAYS_3']}',
            '{row['LEAD_TIME_OVRD_DAYS_4']}',
            '{row['LEAD_TIME_OVRD_DAYS_5']}',
            '{row['LEAD_TIME_OVRD_DAYS_6']}',
            '{row['LEAD_TIME_OVRD_DAYS_7']}',
            '{row['APPLY_RATIONING']}',
            '{row['OP_DAYS_1ST_RUN_1']}',
            '{row['OP_DAYS_1ST_RUN_2']}',
            '{row['OP_DAYS_1ST_RUN_3']}',
            '{row['OP_DAYS_1ST_RUN_4']}',
            '{row['OP_DAYS_1ST_RUN_5']}',
            '{row['OP_DAYS_1ST_RUN_6']}',
            '{row['OP_DAYS_1ST_RUN_7']}',
            '{row['OP_DAYS_2ND_RUN_1']}',
            '{row['OP_DAYS_2ND_RUN_2']}',
            '{row['OP_DAYS_2ND_RUN_3']}',
            '{row['OP_DAYS_2ND_RUN_4']}',
            '{row['OP_DAYS_2ND_RUN_5']}',
            '{row['OP_DAYS_2ND_RUN_6']}',
            '{row['OP_DAYS_2ND_RUN_7']}',
            '{row['SHIP_WITH_VENDOR']}',
            '{row['CONNECTING_VENDOR']}',
            '{row['HAWAII_TRANS_METHOD']}',
            -1,
                TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
                TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            MAIN_FACILITY = EXCLUDED.MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY = EXCLUDED.BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO = EXCLUDED.DAYS_TO_BUILD_TO,
            USE_CYCLE_STOCK_AS_MIN = EXCLUDED.USE_CYCLE_STOCK_AS_MIN,
            USE_FAC_WHSE_SS = EXCLUDED.USE_FAC_WHSE_SS,
            LEAD_TIME_OVRD_DAYS_1 = EXCLUDED.LEAD_TIME_OVRD_DAYS_1,
            LEAD_TIME_OVRD_DAYS_2 = EXCLUDED.LEAD_TIME_OVRD_DAYS_2,
            LEAD_TIME_OVRD_DAYS_3 = EXCLUDED.LEAD_TIME_OVRD_DAYS_3,
            LEAD_TIME_OVRD_DAYS_4 = EXCLUDED.LEAD_TIME_OVRD_DAYS_4,
            LEAD_TIME_OVRD_DAYS_5 = EXCLUDED.LEAD_TIME_OVRD_DAYS_5,
            LEAD_TIME_OVRD_DAYS_6 = EXCLUDED.LEAD_TIME_OVRD_DAYS_6,
            LEAD_TIME_OVRD_DAYS_7 = EXCLUDED.LEAD_TIME_OVRD_DAYS_7,
            APPLY_RATIONING = EXCLUDED.APPLY_RATIONING,
            OP_DAYS_1ST_RUN_1 = EXCLUDED.OP_DAYS_1ST_RUN_1,
            OP_DAYS_1ST_RUN_2 = EXCLUDED.OP_DAYS_1ST_RUN_2,
            OP_DAYS_1ST_RUN_3 = EXCLUDED.OP_DAYS_1ST_RUN_3,
            OP_DAYS_1ST_RUN_4 = EXCLUDED.OP_DAYS_1ST_RUN_4,
            OP_DAYS_1ST_RUN_5 = EXCLUDED.OP_DAYS_1ST_RUN_5,
            OP_DAYS_1ST_RUN_6 = EXCLUDED.OP_DAYS_1ST_RUN_6,
            OP_DAYS_1ST_RUN_7 = EXCLUDED.OP_DAYS_1ST_RUN_7,
            OP_DAYS_2ND_RUN_1 = EXCLUDED.OP_DAYS_2ND_RUN_1,
            OP_DAYS_2ND_RUN_2 = EXCLUDED.OP_DAYS_2ND_RUN_2,
            OP_DAYS_2ND_RUN_3 = EXCLUDED.OP_DAYS_2ND_RUN_3,
            OP_DAYS_2ND_RUN_4 = EXCLUDED.OP_DAYS_2ND_RUN_4,
            OP_DAYS_2ND_RUN_5 = EXCLUDED.OP_DAYS_2ND_RUN_5,
            OP_DAYS_2ND_RUN_6 = EXCLUDED.OP_DAYS_2ND_RUN_6,
            OP_DAYS_2ND_RUN_7 = EXCLUDED.OP_DAYS_2ND_RUN_7,
            SHIP_WITH_VENDOR = EXCLUDED.SHIP_WITH_VENDOR,
            CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
            HAWAII_TRANS_METHOD = EXCLUDED.HAWAII_TRANS_METHOD,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        '''
    )


            cur.execute(
                f'''
                INSERT INTO XXPO_VENDOR_ORDER_INFO (
                    CS_VENDOR_ID,
                    VENDOR_NUMBER,
                    CONNECTING_VENDOR,
                    TRANSIT_DAYS,
                    LAG_DAYS,
                    LEAD_TIME_STATED_TYPE,
                    LEAD_TIME_STATED_DAYS,
                    MINIMUM_BKT_0,
                    MINIMUM_BKT_0_TYPE,
                    MAXIMUM_BKT,
                    MAXIMUM_BKT_TYPE,
                    MIN_BKT_2ND,
                    MIN_BKT_TYPE_2ND,
                    MAX_BKT_2ND,
                    MAX_BKT_TYPE_2ND,
                    FIXED_REVIEW_MON,
                    FIXED_REVIEW_TUE,
                    FIXED_REVIEW_WED,
                    FIXED_REVIEW_THU,
                    FIXED_REVIEW_FRI,
                    FIXED_REVIEW_SAT,
                    FIXED_REVIEW_SUN,
                    VENDOR_SERVICE_LEVEL,
                    DAYS_TO_BUY_OVERRIDE,
                    LOAD_BLDG_LIMIT_FLG,
                    ORDER_FILTER_FLAG,
                    REQUEST_ID,
                    CREATION_DATE,
                    LAST_UPDATE_DATE,
                    CREATED_BY,
                    LAST_UPDATED_BY
                )
                VALUES (
                    {generated_c1},
                    '{row['VENDOR_NBR']}'::numeric,
                    '{row['CONNECTING_VENDOR']}',
                    '{row['TRANSIT_DAYS']}',
                    '{row['LAG_DAYS']}',
                    '{row['LEAD_TIME_CALENDAR_IND']}',
                    '{row['VENDOR_LEAD_DAYS']}',
                    NULLIF('{row['VENDOR_MIN_ORDER_SIZE']}', '')::numeric,
                    '{row['VENDOR_ORDER_UNIT']}',
                    NULLIF('{row['VENDOR_MAX_ORDER_SIZE']}', '')::numeric,
                    '{row['VENDOR_ORDER_UNIT']}',
                    NULLIF('{row['VENDOR_MIN_ORDER2_SIZE']}', '')::numeric,
                    '{row['VENDOR_ORDER2_UNIT']}',
                    NULLIF('{row['VENDOR_MAX_ORDER2_SIZE']}', '')::numeric,
                    '{row['VENDOR_ORDER2_UNIT']}',
                    '{row['VENDOR_DLVR_MONDAY_FLG']}',
                    '{row['VENDOR_DLVR_TUESDAY_FLG']}',
                    '{row['VENDOR_DLVR_WEDNESDAY_FLG']}',
                    '{row['VENDOR_DLVR_THURSDAY_FLG']}',
                    '{row['VENDOR_DLVR_FRIDAY_FLG']}',
                    '{row['VENDOR_DLVR_SATURDAY_FLG']}',
                    '{row['VENDOR_DLVR_SUNDAY_FLG']}',
                    NULLIF('{row['VENDOR_SERVICE_LEVEL']}', '')::numeric,
                    '{row['DAYS_TO_BUY_OVERRIDE']}',
                    '{row['LOAD_BLDG_LIMIT_FLG']}',
                    '{row['FILTER_VENDOR_FLG']}',
                    -1,
                        TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
                        TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
                    '-1',
                    '-1'
                )
                ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
                    VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
                    CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
                    TRANSIT_DAYS = EXCLUDED.TRANSIT_DAYS,
                    LAG_DAYS = EXCLUDED.LAG_DAYS,
                    LEAD_TIME_STATED_TYPE = EXCLUDED.LEAD_TIME_STATED_TYPE,
                    LEAD_TIME_STATED_DAYS = EXCLUDED.LEAD_TIME_STATED_DAYS,
                    MINIMUM_BKT_0 = EXCLUDED.MINIMUM_BKT_0,
                    MINIMUM_BKT_0_TYPE = EXCLUDED.MINIMUM_BKT_0_TYPE,
                    MAXIMUM_BKT = EXCLUDED.MAXIMUM_BKT,
                    MAXIMUM_BKT_TYPE = EXCLUDED.MAXIMUM_BKT_TYPE,
                    MIN_BKT_2ND = EXCLUDED.MIN_BKT_2ND,
                    MIN_BKT_TYPE_2ND = EXCLUDED.MIN_BKT_TYPE_2ND,
                    MAX_BKT_2ND = EXCLUDED.MAX_BKT_2ND,
                    MAX_BKT_TYPE_2ND = EXCLUDED.MAX_BKT_TYPE_2ND,
                    FIXED_REVIEW_MON = EXCLUDED.FIXED_REVIEW_MON,
                    FIXED_REVIEW_TUE = EXCLUDED.FIXED_REVIEW_TUE,
                    FIXED_REVIEW_WED = EXCLUDED.FIXED_REVIEW_WED,
                    FIXED_REVIEW_THU = EXCLUDED.FIXED_REVIEW_THU,
                    FIXED_REVIEW_FRI = EXCLUDED.FIXED_REVIEW_FRI,
                    FIXED_REVIEW_SAT = EXCLUDED.FIXED_REVIEW_SAT,
                    FIXED_REVIEW_SUN = EXCLUDED.FIXED_REVIEW_SUN,
                    VENDOR_SERVICE_LEVEL = EXCLUDED.VENDOR_SERVICE_LEVEL,
                    DAYS_TO_BUY_OVERRIDE = EXCLUDED.DAYS_TO_BUY_OVERRIDE,
                    LOAD_BLDG_LIMIT_FLG = EXCLUDED.LOAD_BLDG_LIMIT_FLG,
                    ORDER_FILTER_FLAG = EXCLUDED.ORDER_FILTER_FLAG,
                    REQUEST_ID = EXCLUDED.REQUEST_ID,
                    CREATION_DATE = EXCLUDED.CREATION_DATE,
                    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                    CREATED_BY = EXCLUDED.CREATED_BY,
                    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                '''
            )



            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_COSTING_INFO (
            CS_VENDOR_ID,
            CASH_DISC_TERMS,
            CASH_DISC_TERMS_DAYS,
            CASH_DISC_TERMS_NET,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            '{row['VENDOR_DISC_PCT']}',
            '{row['VENDOR_DISCOUNT_DAYS_NBR']}',
            '{row['VENDOR_NET_DAYS']}',
            -1,
                TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
                TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            CASH_DISC_TERMS = EXCLUDED.CASH_DISC_TERMS,
            CASH_DISC_TERMS_DAYS = EXCLUDED.CASH_DISC_TERMS_DAYS,
            CASH_DISC_TERMS_NET = EXCLUDED.CASH_DISC_TERMS_NET,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        ''')


            cur.execute(f'''INSERT INTO XXPO_VENDOR_EDI_INFO (CS_VENDOR_ID,
                VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID,
                CREATION_DATE,
                LAST_UPDATE_DATE,
                CREATED_BY,
                LAST_UPDATED_BY
                ) 
                VALUES (
                {generated_c1},
                '{row['VENDOR_CONTINUITY_REPLEN_FLG']}',
                -1,
                    TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
                    TO_TIMESTAMP('{current_timestamp}', 'YYYY-MM-DD HH24:MI:SS')::date,
                '-1',
                '-1') 
                ON CONFLICT (CS_VENDOR_ID) 
                DO UPDATE SET VENDOR_CONTINUITY_REPLEN_FLG = EXCLUDED.VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID = EXCLUDED.REQUEST_ID,
                CREATION_DATE = EXCLUDED.CREATION_DATE,
                LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                CREATED_BY = EXCLUDED.CREATED_BY,
                LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                ''')
            
post_conn.commit()

In [82]:
oracle_df = oracle_df.fillna('')
oracle_df[['VENDOR_TYPE', 'BICEPS_BUYER_CODE',
            'BUYER_NAME',
            'VENDOR_FOODLINK_FLG']]

# '{row['BICEPS_BUYER_CODE']}',
#             '{row['BUYER_NAME']}',
#             '{row['VENDOR_FOODLINK_FLG']}',

,VENDOR_TYPE,BICEPS_BUYER_CODE,BUYER_NAME,VENDOR_FOODLINK_FLG
0,V,J8,,
1,V,J8,,
2,B,J8,,


In [44]:
oracle_df['BUYER_NAME'] 
oracle_df['VENDOR_HOME_LOC_DEST']

0    251.0
1    252.0
2         
Name: VENDOR_HOME_LOC_DEST, dtype: object

In [ ]:
#############################################################################

#############################################################################
# Step 1: Retrieve the last_run from PostgreSQL
cur.execute("SELECT profile_value FROM cspoms_common_profiles WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'")
result = cur.fetchone()
print(f'TIme in the Profile Value',result)

# Retrieve the current timestamp from the database
profile_update_query = "SELECT CURRENT_TIMESTAMP"
cur.execute(profile_update_query)
current_timestamp = cur.fetchone()[0]
print(f'Time it is going to update next',current_timestamp)
# Convert the retrieved value to a datetime object
current_timestamp_dt = datetime.strptime(str(current_timestamp), "%Y-%m-%d %H:%M:%S.%f%z")

# Format the datetime object to match the desired format
current_timestamp_str = current_timestamp_dt.strftime("%Y-%m-%d %H:%M:%S")

if result:
    # Step 2: Convert the retrieved value to a datetime object
    last_run_str = result[0]  # Assuming the result is a string
    # Remove the UTC offset before parsing
    # Remove the UTC offset and microseconds before parsing
    print('last_run_str',last_run_str)
    last_run_str_without_offset = last_run_str.split('+')[0].split('.')[0]
    last_run_datetime = datetime.strptime(last_run_str_without_offset, "%Y-%m-%d %H:%M:%S")
    # Step 3: Format the datetime object to match the Oracle date format
    last_run_formatted = last_run_datetime.strftime("%Y-%m-%d %H:%M:%S")

    # Step 4: Use the formatted date in the Oracle SQL query
    # delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE last_update_date > TO_DATE('{last_run_formatted}', 'YYYY-MM-DD HH24:MI:SS')"
    delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER  WHERE ROWNUM <10"
    delta_df = pd.read_sql(delta_run_query, connection)
    print('@@@@@@@delta_df', delta_df.columns)
    if not delta_df.empty:
        # print('@@@@@@@@@@@@@combined_df,',combined_df.columns)
        delta_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
        # Merge delta_df with combined_df based on 'VENDOR_NBR'
        final_df = pd.merge(combined_df, delta_df, on='VENDOR_NBR', how='outer')
        print('final_df', final_df)
        final_df = pd.merge(combined_df, delta_df, how='left', left_on='bic_vendor_number', right_on='VENDOR_NBR')
    else:
        # print('No rows to fetch in delta logic from the last run')
        final_df = combined_df
        # print('final_df', final_df)
else:
    print("No Profile value in cspoms_common_profiles for I1619_CSPOMS_RLX_BROKER_VENDOR. taking full records")
    oracle_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE ROWNUM <10"

In [58]:
post_cur.execute("SELECT profile_value FROM cspoms_common_profiles WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'")
result = post_cur.fetchone()
print(f'TIme in the Profile Value',result)

# Retrieve the current timestamp from the database
profile_update_query = "SELECT CURRENT_TIMESTAMP"
post_cur.execute(profile_update_query)
current_timestamp = post_cur.fetchone()[0]
print(f'Time it is going to update next',current_timestamp)
# Convert the retrieved value to a datetime object
current_timestamp_dt = datetime.strptime(str(current_timestamp), "%Y-%m-%d %H:%M:%S.%f%z")

# Format the datetime object to match the desired format
current_timestamp_str = current_timestamp_dt.strftime("%Y-%m-%d %H:%M:%S")
# current_timestamp_str
if result:
    # Step 2: Convert the retrieved value to a datetime object
    last_run_str = result[0]  # Assuming the result is a string
    # Remove the UTC offset before parsing
    # Remove the UTC offset and microseconds before parsing
    print('last_run_str',last_run_str)
    last_run_str_without_offset = last_run_str.split('+')[0].split('.')[0]
    last_run_datetime = datetime.strptime(last_run_str_without_offset, "%Y-%m-%d %H:%M:%S")
    # Step 3: Format the datetime object to match the Oracle date format
    last_run_formatted = last_run_datetime.strftime("%Y-%m-%d %H:%M:%S")

    # Step 4: Use the formatted date in the Oracle SQL query
    delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE last_update_date > TO_DATE('{last_run_formatted}', 'YYYY-MM-DD HH24:MI:SS')"
#     delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER  WHERE ROWNUM <10"
    delta_df = pd.read_sql(delta_run_query, connection)
    print('@@@@@@@delta_df', delta_df.columns)
    if not delta_df.empty:
        # print('@@@@@@@@@@@@@combined_df,',combined_df.columns)
        delta_df['VENDOR_NBR'] = delta_df['VENDOR_NBR'].str.zfill(6)
        print(delta_df)
    else:
        # print('No rows to fetch in delta logic from the last run')
        print('No rows to fetch from oracle delta logic')
        # print('final_df', final_df)
else:
    print("No Profile value in cspoms_common_profiles for I1619_CSPOMS_RLX_BROKER_VENDOR. taking full records")
    delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER  WHERE ROWNUM <100"
    delta_df = pd.read_sql(delta_run_query, connection)
print('END')

TIme in the Profile Value ('2024-01-09 14:47:12',)
Time it is going to update next 2024-01-17 13:15:22.661110+00:00
last_run_str 2024-01-09 14:47:12
@@@@@@@delta_df Index(['VENDOR_SITE_ID', 'VENDOR_NBR', 'VENDOR_NAME', 'VENDOR_START_DATE',
       'VENDOR_END_DATE', 'VENDOR_STATUS', 'VENDOR_TYPE', 'AP_VENDOR_ID',
       'AP_VENDOR_NAME', 'AP_VENDOR_ALT_NAME',
       ...
       'PICKUP_DAY_6', 'PICKUP_DAY_7', 'EOQ_CAP_DAYS', 'WHSE_SHIP_TO',
       'SHIP_WITH_VENDOR', 'RANK_PERCENT_OVERRIDE', 'PO_DATE_CHANGE_RSN_FLAG',
       'PULLBKS_AVAILABLE', 'HAWAII_TRANS_METHOD', 'TEST_NEW'],
      dtype='object', length=259)
   VENDOR_SITE_ID VENDOR_NBR              VENDOR_NAME   VENDOR_START_DATE  \
0              49   10000044                     None 2013-10-20 15:09:01   
1         1035031   10000044                     None 2016-07-14 14:02:46   
2         3275707     445932      INTL. TRADE IMPORTS 2022-07-20 00:00:00   
3         3276057     447709  INTNL TRADE IMPORTS LLC 2022-08-05 00:00:0

In [270]:
def fetch_oracle_delta_df(post_cur, connection):
    # Step 1: Retrieve the last_run from PostgreSQL
    post_cur.execute("SELECT profile_value FROM cspoms.cspoms_common_profiles WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'")
    result = post_cur.fetchone()
    print(f'Time in the Profile Value', result)

    # Retrieve the current timestamp from the database
    profile_update_query = "SELECT CURRENT_TIMESTAMP"
    post_cur.execute(profile_update_query)
    current_timestamp = post_cur.fetchone()[0]
#     print(f'Time it is going to update next', current_timestamp)

    # Convert the retrieved value to a datetime object
    current_timestamp_dt = datetime.strptime(str(current_timestamp), "%Y-%m-%d %H:%M:%S.%f%z")

    # Format the datetime object to match the desired format
    current_timestamp_str = current_timestamp_dt.strftime("%Y-%m-%d %H:%M:%S")
    print(f'Time it is going to update next', current_timestamp_str)
    # Initialize the delta_df variable
    oracle_delta_df = pd.DataFrame()

    if result[0] is not None:
        # Step 2: Convert the retrieved value to a datetime object
        last_run_str = result[0]  # Assuming the result is a string

        # Remove the UTC offset before parsing
        last_run_str_without_offset = last_run_str.split('+')[0].split('.')[0]
        last_run_datetime = datetime.strptime(last_run_str_without_offset, "%Y-%m-%d %H:%M:%S")

        # Step 3: Format the datetime object to match the Oracle date format
        last_run_formatted = last_run_datetime.strftime("%Y-%m-%d %H:%M:%S")

        # Step 4: Use the formatted date in the Oracle SQL query
        delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE last_update_date > TO_DATE('{last_run_formatted}', 'YYYY-MM-DD HH24:MI:SS')"
        oracle_delta_df = pd.read_sql(delta_run_query, connection)

        if not oracle_delta_df.empty:
            oracle_delta_df['VENDOR_NBR'] = oracle_delta_df['VENDOR_NBR'].str.zfill(6)
            print(oracle_delta_df)
        else:
            print('No rows to fetch from oracle delta logic')
    else:
        print("No Profile value in cspoms_common_profiles for I1619_CSPOMS_RLX_BROKER_VENDOR. taking full records")
        delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER  WHERE ROWNUM <100"
        oracle_delta_df = pd.read_sql(delta_run_query, connection)

    return oracle_delta_df

# current_timestamp_str = '2024-01-09 14:47:12'
oracle_delta_df = fetch_oracle_delta_df(post_cur, connection)
# oracle_delta_df
if not oracle_delta_df.empty:
    oracle_delta_df = oracle_delta_df.fillna('')
    oracle_delta_df['VENDOR_HOME_LOC_DEST'] = pd.to_numeric(oracle_delta_df['VENDOR_HOME_LOC_DEST'], errors='coerce').astype(int)
    oracle_delta_df.loc[oracle_delta_df['VENDOR_TYPE'] == 'Broker', 'VENDOR_TYPE'] = 'B'
    oracle_delta_df.loc[oracle_delta_df['VENDOR_TYPE'] == 'Buyer Vendor', 'VENDOR_TYPE'] = 'V'
    oracle_delta_df['VENDOR_TYPE'] = oracle_delta_df['VENDOR_TYPE'].str[0]
#     oracle_delta_df = oracle_delta_df.fillna('')
    upsert_vendor_master_and_childs_oracle(oracle_delta_df, post_cur)
    post_cur.execute(f'''UPDATE cspoms.cspoms_common_profiles 
    SET profile_value = '{current_timestamp_str}'
    WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'
    ''')
    print('Updated the current_timestamp_str to the profile value in cspoms_common_profiles')
else:
    print("No data fetched from oracle delta logic. Skipping upsert_vendor_master_and_childs_oracle.")
post_conn.commit()

Time in the Profile Value ('2024-01-17 13:15:22',)
Time it is going to update next 2024-01-24 06:43:06


KeyboardInterrupt: 

In [277]:
oracle_delta_df

,VENDOR_SITE_ID,VENDOR_NBR,VENDOR_NAME,VENDOR_START_DATE,VENDOR_END_DATE,VENDOR_STATUS,VENDOR_TYPE,AP_VENDOR_ID,AP_VENDOR_NAME,AP_VENDOR_ALT_NAME,...,PICKUP_DAY_6,PICKUP_DAY_7,EOQ_CAP_DAYS,WHSE_SHIP_TO,SHIP_WITH_VENDOR,RANK_PERCENT_OVERRIDE,PO_DATE_CHANGE_RSN_FLAG,PULLBKS_AVAILABLE,HAWAII_TRANS_METHOD,TEST_NEW
0,49,10000044,,2013-10-20 15:09:01,2016-07-14 14:02:48,I,PAY SITE,31,UPDATE,HEINZ FROZEN FOOD CO,...,,,,,,,,,,
1,50,10012849,,2013-10-20 15:09:01,2018-05-31 14:21:18,I,PAY SITE,8651,WHINK PRODUCT CO,WHINK PRODUCT CO,...,,,,,,,,,,
2,51,10013959,,2013-10-20 15:09:01,,A,PAY SITE,9359,SAU SEA FOODS INC,SAU SEA FOODS INC,...,,,,,,,,,,
3,52,10021006,,2013-10-20 15:09:01,,A,PAY SITE,13533,M M T A SERVICES,M M T A SERVICES,...,,,,,,,,,,
4,53,10021584,,2013-10-20 15:09:01,,A,PAY SITE,13971,TOPS INC,TOPS INC,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,23,10021927,,2013-10-20 15:09:00,,A,PAY SITE,14285,APPLE & EVE,APPLE & EVE,...,,,,,,,,,,
95,24,10020893,,2013-10-20 15:09:00,,A,PAY SITE,13447,TRIDENT SEAFOODS,TRIDENT SEAFOODS,...,,,,,,,,,,
96,25,10002918,,2013-10-20 15:09:00,,A,PAY SITE,2141,HANOVER FOODS CORPORATION,HANOVER FOODS CORP,...,,,,,,,,,,
97,26,10007801,,2013-10-20 15:09:00,2022-03-08 00:00:00,I,PAY SITE,5374,MILMAR FOOD GROUP LLC,MILMAR FOOD GROUP LLC,...,,,,,,,,,,


In [94]:
for i in oracle_delta_df['AP_VENDOR_NAME']:
    print(i)

UPDATE
WHINK PRODUCT CO
SAU SEA FOODS INC
M M T A SERVICES
TOPS INC
RELIABLE DIESEL
NYS TAX DEPARTMENT COMM OF TAX & FIN
MATPAC INC
CASTROVILLE STATION SHOPPING CENTER
HARRIS TEA CO
NORTH COAST SEAFOODS
HEARTHMARK LLC
VITA FOOD PRODUCTS INC
DURO BAG MFG CO
WASHBURN F B CANDY
SPANGLER CANDY CO
JOHN J DITO & SON
MOUNTAIN STATES ROSEN LLC
ASSOCIATED MILK PROD AMPI-SDS
JOHN MORRELL & CO
ROMALDO MARTIN
CAVENDISH FARMS INC
SPARTAN FOODS OF AMERICA INC SC
MIDDLE EAST BAKERY
JR SIMPLOT COMPANY
CHICK'S FLEET & AUTO ELECTRICAL SERVICE
HARTZ MOUNTAIN CORP
DEL MONTE FOODS GA
OCEAN SPRAY
DOLE PACKAGED FOODS
AUTOCRAT INC
CONAGRA FOOD SALES INC
ABEL WOMACK
S C JOHNSON & SON
BLUE DIAMOND GROWERS
WEIMAN PRODUCTS LLC
PATRICK CUDAHY INC
NESTLE USA INC
DOMINO FOODS  INC
B & R PLASTICS
ATALANTA CORP
HOME MARKET FOOD/FREEZER QUEEN
FERRERO USA INC
RHODES INTERNATIONAL
SFS/BROOKLYN BOTTLING GROUP
POLAR M W  FOODS
BAKE N JOY FOODS INC
3M COMPANY
MILLBROOK DIST SRVCS
EVERGREEN OIL INC
COMDATA
BRATTLEBORO WINTER

In [99]:
post_cur.execute(f'''UPDATE cspoms.cspoms_common_profiles 
    SET profile_value = null
    WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'
    ''')

In [140]:
test_df['BIC_SSA_FLAG']

0    None
1    None
2    None
3    None
4    None
Name: BIC_SSA_FLAG, dtype: object

In [276]:
test_df
merged_df

,STG_SEQ_ID,VENDOR_FACILITY,BIC_VENDOR_NUMBER,BIC_BUYER_NUMBER,BIC_CRP_LEVEL,BIC_STATUS,BIC_PO_RECOMMENDED_IND,BIC_AFE_TYPE1,PO_TRANSMISSION_VALUE,PO_TRANSMISSION_TYPE,...,LOAD_BUILDING,FACILITY_NAME,PREPAYADD_BUY_RATE_CHANGED,BACKHAUL_BUY_RATE_CHANGED,FREIGHT_BILL_BUY_RATE_CHANGED,SSA_RATE_BASIS_GROSS_FLAG,MAXIMUM_BKT_TYPE,MAX_BKT_TYPE_2ND,BACKHAUL_BUY_RATE_TYPE,PRE_PAY_ADD_FLAG
0,,1,,,,,,,,,...,F,C&S BRATTLEBORO,,,,,CS,LB,,
1,,1,,,,,,,,,...,F,C&S BRATTLEBORO,,,,,DOL,LB,,
2,,1,,,,,,,,,...,F,C&S BRATTLEBORO,,,,,PLT,,,
3,,1,,,,,,,,,...,F,C&S BRATTLEBORO,,,,,CS,,,
4,,1,,,,,,,,,...,F,C&S BRATTLEBORO,,,,,DOL,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64700,,,,,,,,,,,...,B,NFR - C&S - Optical Ave,,,,,,,,
64704,,,,,,,,,,,...,B,C&S MAULDIN,,,,,,,,
64710,,,,,,,,,,,...,B,C&S ABERDEEN,,,,,,,,
64712,,,,,,,,,,,...,B,NFR - C&S - Optical Ave,,,,,,,,


In [227]:

test_df = pd.read_sql(oracle_test, connection)
def format_decimal_columns(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].apply(lambda x: f'{float(x):}' if pd.notna(x) and x != '' and pd.to_numeric(x, errors='coerce') % 1 != 0 and not np.isnan(pd.to_numeric(x, errors='coerce')) else f'{int(float(x))}' if pd.notna(x) else x)
    return df
# merged_df = format_decimal_columns(merged_df.astype(str), merged_df.columns)
test_df = format_decimal_columns(test_df)
# test_df = oracle_cur.execute(test_oracle)
test_df.replace({pd.NA: None}, inplace=True)


test_df = test_df.drop_duplicates(subset=['BIC_VENDOR_FACILITY', 'VENDOR_NBR'])
#### 1 processing
test_df['VENDOR_NBR'] = test_df['VENDOR_NBR'].str.zfill(6)
test_df['CREATION_DATE'] = test_df['BIC_CREATION_DATE']
test_df['request_id'] = -999
test_df[['BIC_VENDOR_FACILITY', 'VENDOR_NBR','DIV_VNDR_FLG','VENDOR_FOODLINK_FLG','VENDOR_FREIGHT_FLG']]
final_df = test_df
def determine_load_building(row):
    try:
        if (
            #['DIV_VNDR_FLG','VENDOR_FOODLINK_FLG','VENDOR_FREIGHT_FLG']
            row['DIV_VNDR_FLG'] is not None and not pd.isna(row['DIV_VNDR_FLG']) or
            row['VENDOR_FOODLINK_FLG'] is not None and not pd.isna(row['VENDOR_FOODLINK_FLG']) or
            row['VENDOR_FREIGHT_FLG'] is not None and not pd.isna(row['VENDOR_FREIGHT_FLG'])
        ):
            return 'F'
        else:
            return 'B'
    except Exception as e:
        print(f"Error in determine_load_building function: {e}")
        print(f"Row details: {row}")
        raise  # Reraise the exception to see the traceback

# Apply the custom function to create the LOAD_BUILDING column
final_df['LOAD_BUILDING'] =  final_df.apply(determine_load_building, axis=1)
# Update 'VENDOR_TYPE' column based on conditions
# final_df.loc[ final_df['VENDOR_TYPE'] == 'Broker', 'VENDOR_TYPE'] = 'B'
# final_df.loc[final_df['VENDOR_TYPE'] == 'Buyer Vendor', 'VENDOR_TYPE'] = 'V'
# Handle all the VENDOR_TYPE
# Replace specific values in 'VENDOR_TYPE' with corresponding single characters
final_df['VENDOR_TYPE'].replace({
    'Sub Vendor': 'S',
    'Broker': 'B',
    'Buyer Vendor': 'V',
    'DIST PARTNER': 'D',
    'DESTINATION': 'O',  # Replace with the desired character
    'PAY SITE': 'P',
    np.nan: 'N'  # Replace NaN with 'N' or any other character
}, inplace=True)
# Duplicate the 'SHIP_TO_LOCATION_NAME' column as 'FACILITY_NAME'
final_df = final_df.assign(FACILITY_NAME=final_df['SHIP_TO_LOCATION_NAME'])
# CREATION_DATE	 :  	PREPAYADD_BUY_RATE_CHANGED
final_df = final_df.assign(PREPAYADD_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
final_df = final_df.assign(BACKHAUL_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
final_df = final_df.assign(FREIGHT_BILL_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
# BIC_SSA_FLAG	 : 	SSA_RATE_BASIS_GROSS_FLAG
final_df = final_df.assign(SSA_RATE_BASIS_GROSS_FLAG= final_df['BIC_SSA_FLAG'])
final_df = final_df.assign(MAXIMUM_BKT_TYPE= final_df['VENDOR_ORDER_UNIT'])
#     'VENDOR_ORDER2_UNIT':'MAX_BKT_TYPE_2ND'
final_df = final_df.assign(MAX_BKT_TYPE_2ND= final_df['VENDOR_ORDER2_UNIT'])
# 	,'bic_backhaul_type'	 : 	'BACKHAUL_BUY_RATE_TYPE'
final_df = final_df.assign(BACKHAUL_BUY_RATE_TYPE= final_df['BIC_BACKHAUL_TYPE'])
#     ,'bic_flag_prepay_and_add'	 : 	'PRE_PAY_ADD_FLAG'
final_df = final_df.assign(PRE_PAY_ADD_FLAG= final_df['BIC_FLAG_PREPAY_AND_ADD'])
############################################################
#Drop the column before renaming
############################################################
# List of columns to drop (including one that doesn't exist)
columns_to_drop = ['OTHER_COLUMN_1', 'OTHER_COLUMN_2', 'NON_EXISTENT_COLUMN','VENDOR_FACILITY']
# Drop the specified columns with errors='ignore'
final_df = final_df.drop(columns=columns_to_drop, errors='ignore')
            # # Rename columns
merged_df = final_df.rename(columns={
    'VENDOR_SITE_ID': 'CS_VEND_SITE_ID',
    'VENDOR_NBR': 'VENDOR_NUMBER',
    'BUYER_VENDOR_NAME': 'VENDOR_NAME',
    'BIC_VENDOR_FACILITY': 'VENDOR_FACILITY',
    'BICEPS_BUYER_CODE': 'BUYER_ID',
    'BIC_WAREHOUSE_CODE': 'WAREHOUSE_CODE',
    'VENDOR_FOODLINK_FLG': 'FOOD_LINK_FLAG',
    'VENDOR_BROKER_NBR': 'BROKER_NUMBER',
    'VENDOR_BROKER_NAME': 'BROKER_NAME',
    'BIC_FLAG_TRUCK': 'TRUCK_FLAG',
    'BIC_FLAG_RAIL': 'RAIL_FLAG',
    'BIC_AFE_CONTACT': 'PO_TRANSMISSION_TYPE',  
    'BIC_AFE_FAX_NUMBER': 'PO_TRANSMISSION_VALUE',
    'BIC_FREIGHT_ALLOW': 'FREIGHT_ALLOW_BUY_RATE',
    'LEAD_TIME_CALENDAR_IND' : 'LEAD_TIME_STATED_TYPE',
    'VENDOR_LEAD_DAYS' : 'LEAD_TIME_STATED_DAYS',
    'VENDOR_MIN_ORDER_SIZE' : 'MINIMUM_BKT_0',
    'VENDOR_ORDER_UNIT' : 'MINIMUM_BKT_0_TYPE',
    'VENDOR_MAX_ORDER_SIZE' : 'MAXIMUM_BKT',
    'VENDOR_MIN_ORDER2_SIZE':'MIN_BKT_2ND',
    'VENDOR_ORDER2_UNIT':'MIN_BKT_TYPE_2ND',
    'VENDOR_MAX_ORDER2_SIZE':'MAX_BKT_2ND'
    ,'VENDOR_DLVR_MONDAY_FLG' :  'FIXED_REVIEW_MON'
    ,'VENDOR_DLVR_TUESDAY_FLG' :  'FIXED_REVIEW_TUE'
    ,'VENDOR_DLVR_WEDNESDAY_FLG' :  'FIXED_REVIEW_WED'
    ,'VENDOR_DLVR_THURSDAY_FLG' :  'FIXED_REVIEW_THU'
    ,'VENDOR_DLVR_FRIDAY_FLG' :  'FIXED_REVIEW_FRI'
    ,'VENDOR_DLVR_SATURDAY_FLG' :  'FIXED_REVIEW_SAT'
    ,'VENDOR_DLVR_SUNDAY_FLG' :  'FIXED_REVIEW_SUN'
    ,'BIC_ORDER_SEQUENCE' : 'ORDER_SEQUENCE'
    ,'FILTER_VENDOR_FLG' : 'ORDER_FILTER_FLAG'
    ,'BIC_FREIGHT_ALLOW'	 :  	'FREIGHT_ALLOW_BUY_RATE'
    ,'BIC_FREIGHT_ALLOW_TYPE'	 :  	'FREIGHT_ALLOW_BUY_RATE_TYPE'
    ,'CREATION_DATE'	 :  	'FREIGHT_ALLOW_BUY_RATE_CHANGED'
    ,'BIC_BACKHAUL_TYPE'	 :  	'PREPAYADD_BUY_RATE_TYPE'
    ,'BIC_FREIGHTBILL'	 : 'FREIGHT_BILL_BUY_RATE'
    ,'BIC_FREIGHTBILL_TYPE'	 : 	'FREIGHT_BILL_BUY_RATE_TYPE'
    ,'BIC_BACKHAUL'	 : 	'BACKHAUL_BUY_RATE'
    ,'BIC_SSA_FLAG'	 : 	'SSA_RATE_BASIS_NET_FLAG'
    ,'BIC_FLAG_PREPAID'	 : 	'PRE_PAY_FLAG'
    ,'BIC_FLAG_FREIGHT_BILL'	 : 	'FREIGHT_BILL_FLAG'
    ,'BIC_FLAG_BACKHAUL'	 : 	'BACKHAUL_FLAG'
    ,'BIC_EXTERNAL_BACKHAUL'	 : 	'EXTERNAL_BACKHAUL_FLAG'
    ,'BIC_SSA_ALLOW_PERCENT'	 : 	'SSA_RATE'
    ,'BIC_SSA_ALLOW_BASIS'	 : 	'SSA_RATE_TYPE'
    ,'BIC_SSA_ALLOW_ACCOUNT'	 : 	'SSA_RATE_BASIS_ACC'
    ,'VENDOR_DISC_PCT'	 : 	'CASH_DISC_TERMS'
    ,'VENDOR_DISCOUNT_DAYS_NBR'	 : 	'CASH_DISC_TERMS_DAYS'
    ,'VENDOR_NET_DAYS'	 : 	'CASH_DISC_TERMS_NET'
})



# def format_decimal_columns(df):
#     for col in df.columns:
#         df[col] = df[col].apply(lambda x: f'{float(x):.3f}' if pd.notna(x) and pd.to_numeric(x, errors='coerce') % 1 != 0 and not np.isnan(pd.to_numeric(x, errors='coerce')) else f'{int(float(x))}' if pd.notna(x) else x)
#     return df
# merged_df = format_decimal_columns(merged_df.astype(str), merged_df.columns)
# merged_df = format_decimal_columns(merged_df)
# merged_df = merged_df.drop_duplicates(subset=['VENDOR_FACILITY', 'VENDOR_NUMBER'])
merged_df['VENDOR_TYPE']
merged_df['VENDOR_FACILITY'] = merged_df['VENDOR_FACILITY'].fillna(merged_df['VENDOR_HOME_LOC_DEST'])
merged_df = merged_df.fillna('')

merged_df[['VENDOR_FACILITY', 'VENDOR_NUMBER','BIC_VENDOR_NUMBER','VENDOR_HOME_LOC_DEST']]
# upsert_vendor_master_and_childs(merged_df,post_cur)
# # # final_df
# post_conn.commit()

,VENDOR_FACILITY,VENDOR_NUMBER,BIC_VENDOR_NUMBER,VENDOR_HOME_LOC_DEST
0,1,000001,,1
1,1,000010,,1
2,1,010001,,1
3,1,100010,,1
4,1,010004,,1
...,...,...,...,...
64700,,099224,,
64704,,996024,,
64710,,998710,,
64712,,999965536,,


In [275]:
upsert_vendor_master_and_childs(merged_df,post_cur)
# # # final_df
# post_conn.commit()

ProgrammingError: can't adapt type 'Series'

In [274]:
upsert_vendor_master_and_childs(merged_df[:1],post_cur)
# # # final_df
post_conn.commit()

ProgrammingError: can't adapt type 'Series'

In [256]:
print(merged_df[merged_df.applymap(lambda x: isinstance(x, pd.Series)).any(axis=1)])
# print(merged_df.head())
merged_df[:5]

Empty DataFrame
Columns: [STG_SEQ_ID, VENDOR_FACILITY, BIC_VENDOR_NUMBER, BIC_BUYER_NUMBER, BIC_CRP_LEVEL, BIC_STATUS, BIC_PO_RECOMMENDED_IND, BIC_AFE_TYPE1, PO_TRANSMISSION_VALUE, PO_TRANSMISSION_TYPE, BIC_MINIMUM_QUANTITY, BIC_MINIMUM_QUANTITY_SIGN, BIC_MINIMUM_TYPE, BIC_BRACKET_QUANTITY, BIC_BRACKET_QUANTITY_SIGN, BIC_CURRENT_BKT_NUMBER, BIC_CURRENT_BKT_QUANTITY, BIC_CURRENT_BKT_QUANTITY_SIGN, BIC_CURRENT_BKT_TYPE, BIC_MAXIMUM_QUANTITY, BIC_MAXIMUM_QUANTITY_SIGN, BIC_MAXIMUM_TYPE, BIC_ORDER_INTERVAL_WEEKS, BIC_ORDER_INTERVAL_WEEKS_SIGN, BIC_LEAD_TIME_STATED_WEEKS, BIC_LEAD_TIME_STATED_WKS_SIGN, BIC_FIXED_REVIEW_DAY1, BIC_FIXED_REVIEW_DAY2, BIC_FIXED_REVIEW_DAY3, BIC_FIXED_REVIEW_DAY4, BIC_FIXED_REVIEW_DAY5, BIC_FIXED_REVIEW_DAY6, BIC_FIXED_REVIEW_DAY7, BIC_TARGET_SERVICE_LEVEL, BIC_TARGET_SERVICE_LEVEL_SIGN, BIC_NAME, BIC_ADDRESS_1, BIC_CITY, BIC_STATE, BIC_ZIP, BIC_PHONE, BIC_CONTACT, BIC_TRANSFER_MATCH_TYPE, BIC_PAYABLE_VENDOR_NBR, BIC_LEAD_TIME_WEEKS_TYPE, BIC_DUNS_REMIT_NO, BIC_

In [176]:
import pandas as pd

def format_decimal_columns(df, columns):
    for col in columns:
        df[col] = df[col].apply(lambda x: f'{x:}' if pd.notna(x) and x % 1 != 0 else f'{int(x)}' if pd.notna(x) else x)
    return df

# Example usage
data_df = pd.DataFrame({
    'Attribute1': [345.0, 345.05, 35.324535],
    'Attribute2': [456.0, 789.123, 123.0]
})

columns_to_format = ['Attribute1', 'Attribute2']
data_df = format_decimal_columns(data_df, columns_to_format)

print(data_df)


  Attribute1 Attribute2
0        345        456
1     345.05    789.123
2  35.324535        123


In [ ]:
oracle_test = '''
WITH numbered_rows AS (
    SELECT
        vm.*,
        bic.*,
                bic.CREATION_DATE AS BIC_CREATION_DATE,

        ROW_NUMBER() OVER (PARTITION BY vm.vendor_home_loc_dest, bic.bic_vendor_facility, vm.vendor_nbr, bic.bic_vendor_number ORDER BY bic.stg_seq_id) AS rn
    FROM
        apps.xxap_vendor_master vm
        LEFT JOIN apps.xxpo_orcl2biceps_vend_stg bic 
        ON CASE WHEN LENGTH(vm.vendor_nbr) < 6 THEN LPAD(vm.vendor_nbr, 6, '0') ELSE vm.vendor_nbr END = bic.bic_vendor_number(+)
    WHERE
    --vm.LAST_UPDATE_DATE > SYSDATE - 50
    vm.vendor_nbr IN ('8571', '445932', '447709', '10000044') 
) 
SELECT
STG_SEQ_ID                          
,BIC_VENDOR_FACILITY                 
,BIC_VENDOR_NUMBER                   
,BIC_BUYER_NUMBER                    
,BIC_CRP_LEVEL   
,BIC_STATUS                          
,BIC_PO_RECOMMENDED_IND              
,BIC_AFE_TYPE1                       
,BIC_AFE_FAX_NUMBER                  
,BIC_AFE_CONTACT                     
,BIC_MINIMUM_QUANTITY                
,BIC_MINIMUM_QUANTITY_SIGN           
,BIC_MINIMUM_TYPE                    
,BIC_BRACKET_QUANTITY                
,BIC_BRACKET_QUANTITY_SIGN           
,BIC_CURRENT_BKT_NUMBER              
,BIC_CURRENT_BKT_QUANTITY            
,BIC_CURRENT_BKT_QUANTITY_SIGN       
,BIC_CURRENT_BKT_TYPE                
,BIC_MAXIMUM_QUANTITY                
,BIC_MAXIMUM_QUANTITY_SIGN           
,BIC_MAXIMUM_TYPE                    
,BIC_ORDER_INTERVAL_WEEKS            
,BIC_ORDER_INTERVAL_WEEKS_SIGN       
,BIC_LEAD_TIME_STATED_WEEKS          
,BIC_LEAD_TIME_STATED_WKS_SIGN       
,BIC_FIXED_REVIEW_DAY1               
,BIC_FIXED_REVIEW_DAY2               
,BIC_FIXED_REVIEW_DAY3               
,BIC_FIXED_REVIEW_DAY4               
,BIC_FIXED_REVIEW_DAY5               
,BIC_FIXED_REVIEW_DAY6               
,BIC_FIXED_REVIEW_DAY7               
,BIC_TARGET_SERVICE_LEVEL            
,BIC_TARGET_SERVICE_LEVEL_SIGN       
,BIC_NAME                            
,BIC_ADDRESS_1                       
,BIC_CITY                            
,BIC_STATE                           
,BIC_ZIP                             
,BIC_PHONE                           
,BIC_CONTACT                         
,BIC_TRANSFER_MATCH_TYPE             
,BIC_PAYABLE_VENDOR_NBR              
,BIC_LEAD_TIME_WEEKS_TYPE            
,BIC_DUNS_REMIT_NO                   
,BIC_DUNS_REMIT_SUFFIX               
,BIC_FACILITY_SHIP_TO                
,BIC_PRE_SCHED_FLAG                  
,BIC_BROKER_NUMBER                   
,BIC_FREIGHT_ALLOW                   
,BIC_FREIGHT_ALLOW_SIGN              
,BIC_FREIGHT_ALLOW_TYPE              
,BIC_TERMS_PERCENT                   
,BIC_TERMS_PERCENT_SIGN              
,BIC_TERMS_DAYS                      
,BIC_TERMS_DAYS_SIGN                 
,BIC_TERMS_NET_DAYS                  
,BIC_TERMS_NET_DAYS_SIGN             
,BIC_TERMS_BASE                      
,BIC_BILL_FOR_SHORT_DEALS_FLAG       
,BIC_TRANSIT_DAYS                    
,BIC_DUNS_NO                         
,BIC_DUNS_SUFFIX                     
,BIC_VENDOR_TYPE                     
,BIC_2ND_MIN_QUANTITY                
,BIC_2ND_MIN_QUANTITY_SIGN           
,BIC_2ND_MIN_TYPE                    
,BIC_2ND_MAX_QUANTITY                
,BIC_2ND_MAX_QUANTITY_SIGN           
,BIC_2ND_MAX_TYPE                    
,BIC_FLAG_PREPAID                    
,BIC_FLAG_PREPAY_AND_ADD             
,BIC_FLAG_FREIGHT_BILL               
,BIC_FLAG_BACKHAUL                   
,BIC_FLAG_TRUCK                      
,BIC_FLAG_RAIL                       
,BIC_BACKHAUL                        
,BIC_BACKHAUL_SIGN                   
,BIC_BACKHAUL_TYPE                   
,BIC_FREIGHTBILL                     
,BIC_FREIGHTBILL_SIGN                
,BIC_FREIGHTBILL_TYPE                
,BIC_ORDER_SEQUENCE                  
,BIC_ORDER_SEQUENCE_SIGN             
,BIC_SSA_ALLOW_PERCENT               
,BIC_SSA_ALLOW_PERCENT_SIGN          
,BIC_SSA_ALLOW_BASIS                 
,BIC_SSA_ALLOW_ACCOUNT               
,BIC_TRANSACTION_ID                  
,BIC_SSA_FLAG                        
,BIC_COUNTRY                         
,BIC_FILTER_VENDOR_FLAG              
,BIC_LOAD_BLDG_LIMIT_FLAG            
,BIC_WAREHOUSE_CODE                  
,BROKER_PROCESS_STATUS               
,BV_PROCESS_STATUS       
,ERROR_MESSAGE

                  
,ATTRIBUTE_CATEGORY                  
,ATTRIBUTE1                          
,ATTRIBUTE2                          
,ATTRIBUTE3                          
,ATTRIBUTE4                          
,ATTRIBUTE5                          
,ATTRIBUTE6                          
,ATTRIBUTE7                          
,ATTRIBUTE8                          
,ATTRIBUTE9                          
,ATTRIBUTE10                         
,ATTRIBUTE11                         
,ATTRIBUTE12                         
,ATTRIBUTE13                         
,ATTRIBUTE14                         
,ATTRIBUTE15                         
,BIC_EXTERNAL_BACKHAUL
,VENDOR_SITE_ID                  
,VENDOR_NBR                      
,VENDOR_NAME                     
,VENDOR_START_DATE               
,VENDOR_END_DATE                 
,VENDOR_STATUS                   
,VENDOR_TYPE                     
,AP_VENDOR_ID                    
,AP_VENDOR_NAME                  
,AP_VENDOR_ALT_NAME              
,AP_VENDOR_NBR                   
,AP_VENDOR_START_DATE            
,AP_VENDOR_END_DATE              
,AP_VENDOR_STATUS                
,COMPANY                         
,SUPPLIER_TYPE                   
,CONNECTING_VENDOR               
,BUYER_VENDOR                    
,BUYER_VENDOR_NAME               
,PAY_SITE_FLAG                   
,PURCHASING_SITE_FLAG            
,PRIMARY_PAY_SITE_FLAG           
,WHSE_VENDOR_NBR                 
,VENDOR_HOME_LOC_DEST            
,SHIP_TO_LOCATION_NAME           
,BILL_TO_LOCATION_NAME           
,VENDOR_SERVICE_LEVEL            
,VENDOR_MULTIPLE_ORDER           
,VENDOR_IMPLUS_FLG               
,VENDOR_TRANSPORT_METHOD_CODE    
,VENDOR_CONTINUITY_REPLEN_FLG    
,VENDOR_EMERG_LEAD_TIME_DAYS     
,VENDOR_INTRANET_BASED_BUY_FLG   
,VENDOR_DISCON_FLG               
,VENDOR_PAY_TERM                 
,VENDOR_CASH_DISC                
,VENDOR_DISC_PCT                 
,VENDOR_DISCOUNT_DAYS_NBR        
,VENDOR_NET_DAYS                 
,VENDOR_EDI_CUTOFF_TIME          
,PREFERRED_VENDOR_IND            
,ES3_VENDOR_FLG                  
,MDSE_BUYER_NBR                  
,VENDOR_FOODLINK_FLG             
,VENDOR_FREIGHT_FLG              
,LEAD_TIME_CALENDAR_IND          
,VENDOR_LEAD_DAYS                
,VENDOR_LEAD_TIME                
,VENDOR_EMRG_LEAD_TIME_DAYS      
,VENDOR_REAL_LEAD_TIME           
,VENDOR_ROUNDING_FLG             
,VENDOR_DLVR_SUNDAY_FLG          
,VENDOR_DLVR_MONDAY_FLG          
,VENDOR_DLVR_TUESDAY_FLG         
,VENDOR_DLVR_WEDNESDAY_FLG       
,VENDOR_DLVR_THURSDAY_FLG        
,VENDOR_DLVR_FRIDAY_FLG          
,VENDOR_DLVR_SATURDAY_FLG        
,VENDOR_ORDER_UNIT               
,VENDOR_MIN_ORDER_SIZE           
,VENDOR_ALT_MIN_ORDER_SIZE       
,VENDOR_MAX_ORDER_SIZE           
,VENDOR_ORDER2_UNIT              
,VENDOR_MIN_ORDER2_SIZE          
,VENDOR_MAX_ORDER2_SIZE          
,VENDOR_ORDER3_UNIT              
,VENDOR_MIN_ORDER3_SIZE          
,VENDOR_MAX_ORDER3_SIZE          
,VENDOR_BROKER_NBR               
,VENDOR_BROKER_NAME              
,VENDOR_CROSSROADS_GROUP         
,PERISHABLE_BROKER_INFO          
,VENDOR_CONTACT_NAME             
,ADDR_LINE1                      
,ADDR_LINE2                      
,ADDR_LINE3                      
,ADDR_LINE4                      
,CITY                            
,STATE                           
,ZIP_CODE                        
,COUNTY                          
,PROVINCE                        
,COUNTRY                         
,AREA_CODE                       
,PHONE_NBR                       
,FAX_AREA_CODE                   
,FAX_NBR                         
,EMAIL_ADDR                      
,VENDOR_BUYER_NBR                
,BICEPS_BUYER_CODE               
,BUYER_NAME
               
,CONNECTING_VENDOR_NAME          
,MDSE_BUYER_NAME                 
,SSA_BASIS                       
,LEGACY_VENDOR_NUM               
,BV_DISCONT_STATUS               
,EDI_COMPLIANCE_FEE              
,VENDOR_ONE_SOURCE_FLG           
,VENDOR_MEMO_FIELD               
,DVC_VENDOR_FLG                  
,LOST_INCOME_POTENTIAL           
,VENDOR_CROSSROADS_FLG           
,VENDOR_CROSSROADS_DATE          
,VENDOR_TOLRNCE                  
,VENDOR_PROGRAM_FLG              
,DIV_VNDR_FLG                    
,FAX_NUM_EXTN                    
,VENDOR_WORK_PHONE_NO            
,VENDOR_WORK_PHONE_EXTN          
,VENDOR_CELL_NO                  
,VENDOR_CELL_EXTN                
,CLAIM_BACKUP_TYPE               
,SSVC                            
,PROC_VENDOR_TYPE                
,VENDOR_FLG2                     
,FILTER_VENDOR_FLG               
,IMPLUS_TARGET_LVL               
,DAYS_TO_BUY_OVERRIDE            
,VENDOR_SUB_FLG                  
,VENDOR_MAX_ALLOW_SUPPLY         
,VENDOR_SUBCLASS                 
,FULL_PALLET_ORDERING_ONLY       
,DUCS_CONVERSION_CODE            
,VENDOR_FREE_ON_BOARD_POINT      
,VENDOR_STD_ORD_INT_NBR          
,AD_FEATH_2_PARM_FLG             
,LOAD_BLDG_LIMIT_FLG             
,D2S_VENDOR_FLG                  
,VENDOR_ARE_DIFF_DESTS_OK        
,VENDOR_MIL_SOLD_TO_ACCT_NBR     
,VENDOR_RDS_PRE_SCHLED_FLG       
,FM_VENDOR_NBR                   
,LOAD_TYPE                       
,ALL_ORDER                       
,VENDOR_LOAD_TIME_DAYS           
,VENDOR_ORIG_CONTACT_INFO_ID     
,VENDOR_ZONE                     
,REDUCE_RETAIL_EXT_FLG           
,UCS875_MCASE_COST               
,VENDOR_BD_COUNTER               
,VENDOR_INACTIVATE_USER          
,VENDOR_DISCONT_DATE             
,PARENT_VENDOR                   
,BACKHAUL_DAYS_TRANSIT           
,BACKHAUL_PICKUP_POINT           
,BACKHUAL_ADDRESS                
,BACKHAUL_CITY                   
,BACKHAUL_STATE                  
,BACKHAUL_TELEPHONE_NBR          
,BACKHAUL_EXTN                   
,BACKHAUL_HRS                    
,BD_FACTOR                       
,SPECIAL_OFF_INVOICE             
,EDI_PARTNER_ID                  
,EDI_VERSION                     
,VENDOR_DUNS_NBR                 
,UCS880_TRADING_PARTNER          
,UCS852_FLG                      
,UCS852_OCCURRENCE_FLG           
,UCS852_TRADING_PARTNER          
,MIL_DUNSNBR_FLG                 
,DUCS_BROKER_ID                  
,ES3_VENDOR_DUNS_NBR             
,VENDOR_ALLOW_TRANS              
,EDI852_PARTNER_NAME             
,VENDOR_DEST_NBR                 
,DEFAULT_DUNS_NBR                
,VMI_852_855_VERSION             
,DUCS_ACTIVE_FLG                 
,UCS867_FLG                      
,UCS852_GIS_FLG                  
,EDI_DEFAULT_DEST                
,DUCS_850_PO_FLG                 
,VENDOR_3PL_FLG                  
,VENDOR_3PL_CHAIN_NBR            
,VENDOR_3PL_CUST_VENDOR_NBR      
,GROUP_OWNER                     
,GL_CODE                         
,MANF_UPC                        
,VENDOR_DEFAULT_DEST_MER         
,VENDOR_CONSLTD_FLG              
,VNDR_COMM_MTHD                  
,VNDR_COMM_EMAIL                
,VNDR_COMM_FAX                   
,PWMW_BV_NAME                    
,PWMW_AP_NUM                     
,PWMW_AP_NAME                    
,PWMW_WHS_NUM                    
,PWMW_ITEM_DEPT                  
,VENDOR_FACILITY                 
,BICEPS_PO_TYPE                  
,ELIGIBLE_FOR_EMPTY_PO           
,OPPRTSTIC_DPS                   
,BUILD_TO_DAYS_SUPPLY            
,DAYS_TO_BUILD_TO                
,USE_CYCLE_STOCK_AS_MIN          
,TRANSIT_DAYS                    
,LAG_DAYS                        
,BICEPS_VENDOR_TYPE              
,ZERO_SAFETY_STOCK               
,USE_FAC_WHSE_SS                 
,APPLY_RATIONING                 
,DEMAND_DAYS                     
,OP_DAYS_SHIPPER                 
,OP_DAYS_1ST_RUN_1               
,OP_DAYS_1ST_RUN_2               
,OP_DAYS_1ST_RUN_3               
,OP_DAYS_1ST_RUN_4               
,OP_DAYS_1ST_RUN_5               
,OP_DAYS_1ST_RUN_6               
,OP_DAYS_1ST_RUN_7               
,OP_DAYS_2ND_RUN_1               
,OP_DAYS_2ND_RUN_2               
,OP_DAYS_2ND_RUN_3               
,OP_DAYS_2ND_RUN_4               
,OP_DAYS_2ND_RUN_5               
,OP_DAYS_2ND_RUN_6               
,OP_DAYS_2ND_RUN_7               
,MAIN_FACILITY                   
,LEAD_TIME_OVRD_DAYS_1           
,LEAD_TIME_OVRD_DAYS_2           
,LEAD_TIME_OVRD_DAYS_3           
,LEAD_TIME_OVRD_DAYS_4           
,LEAD_TIME_OVRD_DAYS_5           
,LEAD_TIME_OVRD_DAYS_6           
,LEAD_TIME_OVRD_DAYS_7           
,SIMULTD_BUYING_VENDOR           
,SIMULTD_BUYING_SPLIT            
,SIMULTD_APPR_SAT                
,SIMULTD_APPR_SUN                
,SIMULTD_APPR_HOLIDAY            
,COLBRTV_ORD_BLDG                
,SWELL_ALLOW_PCT                 
,SWELL_ALLOW_BASIS               
,SWELL_ALLOW_ACCT                
,PICK_SLOT_CPTY_ELIGIBLE         
,LOGSTCL_PROGRAM_FLAG            
,LOGSTCL_PROGRAM_PCT             
,PICKUP_DAY_1                    
,PICKUP_DAY_2                    
,PICKUP_DAY_3                    
,PICKUP_DAY_4                    
,PICKUP_DAY_5                    
,PICKUP_DAY_6                    
,PICKUP_DAY_7                    
,EOQ_CAP_DAYS                    
,WHSE_SHIP_TO                    
,SHIP_WITH_VENDOR                
,RANK_PERCENT_OVERRIDE           
,PO_DATE_CHANGE_RSN_FLAG         
,PULLBKS_AVAILABLE                            
,BIC_CREATION_DATE             
      
FROM
    numbered_rows
WHERE
    rn = 1


'''

In [266]:
def upsert_vendor_master_and_childs(merged_df,cur):
    # Iterate over DataFrame rows for upsert
    for index, row in merged_df.iterrows():
        # cur.execute("SELECT nextval('cs_vendor_id_seq')")
        # next_sequence_value = cur.fetchone()[0]
        
        # print(next_sequence_value)
        cur.execute(
        f'''INSERT INTO cspoms.XXPO_VENDOR_MASTER (
            CS_VENDOR_ID,
            VENDOR_NUMBER,
            VENDOR_FACILITY,
            CS_VEND_SITE_ID,
            CONNECTING_VENDOR,
            VENDOR_NAME,
            VENDOR_TYPE,
            FACILITY_NAME,
            VENDOR_STATUS,
            AP_VENDOR_NBR,
            AP_VENDOR_NAME,
            BUYER_ID,
            BUYER_NAME,
            LOAD_BUILDING,
            WAREHOUSE_CODE,
            FOOD_LINK_FLAG,
            ADDR_LINE1,
            ADDR_LINE2,
            ADDR_LINE3,
            CITY,
            STATE,
            ZIP_CODE,
            COUNTRY,
            PHONE_NBR,
            VENDOR_CONTACT_NAME,
            SHIP_TO_LOCATION_NAME,
            BILL_TO_LOCATION_NAME,
            BROKER_NUMBER,
            BROKER_NAME,
            TRUCK_FLAG,
            RAIL_FLAG,
            PO_TRANSMISSION_TYPE,
            PO_TRANSMISSION_VALUE,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        ) 
        VALUES (
            
             nextval('cs_vendor_id_seq'),
            '{row['VENDOR_NUMBER']}'::numeric,
            '{row['VENDOR_FACILITY']}',
            '{row['CS_VEND_SITE_ID']}',
            '{row['CONNECTING_VENDOR']}',
            '{row['VENDOR_NAME']}',
            '{row['VENDOR_TYPE']}',
            '{row['FACILITY_NAME']}',
            '{row['VENDOR_STATUS']}',
            '{row['AP_VENDOR_NBR']}',
            '{row['AP_VENDOR_NAME']}',
            '{row['BUYER_ID']}',
            '{row['BUYER_NAME']}',
            '{row['LOAD_BUILDING']}',
            '{row['WAREHOUSE_CODE']}',
            '{row['FOOD_LINK_FLAG']}',
            '{row['ADDR_LINE1']}',
            '{row['ADDR_LINE2']}',
            '{row['ADDR_LINE3']}',
            '{row['CITY']}',
            '{row['STATE']}',
            '{row['ZIP_CODE']}',
            '{row['COUNTRY']}',
            '{row['PHONE_NBR']}',
            '{row['VENDOR_CONTACT_NAME']}',
            '{row['SHIP_TO_LOCATION_NAME']}',
            '{row['BILL_TO_LOCATION_NAME']}',
            '{row['BROKER_NUMBER']}',
            '{row['BROKER_NAME']}',
            '{row['TRUCK_FLAG']}',
            '{row['RAIL_FLAG']}',
            '{row['PO_TRANSMISSION_TYPE']}',
            '{row['PO_TRANSMISSION_VALUE']}',
            '{row['request_id']}',
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'


        ) 
        ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET 
            VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
            VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY,
            CS_VEND_SITE_ID = EXCLUDED.CS_VEND_SITE_ID,
            CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
            VENDOR_NAME = EXCLUDED.VENDOR_NAME,
            VENDOR_TYPE = EXCLUDED.VENDOR_TYPE,
            FACILITY_NAME = EXCLUDED.FACILITY_NAME,
            VENDOR_STATUS = EXCLUDED.VENDOR_STATUS,
            AP_VENDOR_NBR = EXCLUDED.AP_VENDOR_NBR,
            AP_VENDOR_NAME = EXCLUDED.AP_VENDOR_NAME,
            BUYER_ID = EXCLUDED.BUYER_ID,
            BUYER_NAME = EXCLUDED.BUYER_NAME,
            LOAD_BUILDING = EXCLUDED.LOAD_BUILDING,
            WAREHOUSE_CODE = EXCLUDED.WAREHOUSE_CODE,
            FOOD_LINK_FLAG = EXCLUDED.FOOD_LINK_FLAG,
            ADDR_LINE1 = EXCLUDED.ADDR_LINE1,
            ADDR_LINE2 = EXCLUDED.ADDR_LINE2,
            ADDR_LINE3 = EXCLUDED.ADDR_LINE3,
            CITY = EXCLUDED.CITY,
            STATE = EXCLUDED.STATE,
            ZIP_CODE = EXCLUDED.ZIP_CODE,
            COUNTRY = EXCLUDED.COUNTRY,
            PHONE_NBR = EXCLUDED.PHONE_NBR,
            VENDOR_CONTACT_NAME = EXCLUDED.VENDOR_CONTACT_NAME,
            SHIP_TO_LOCATION_NAME = EXCLUDED.SHIP_TO_LOCATION_NAME,
            BILL_TO_LOCATION_NAME = EXCLUDED.BILL_TO_LOCATION_NAME,
            BROKER_NUMBER = EXCLUDED.BROKER_NUMBER,
            BROKER_NAME = EXCLUDED.BROKER_NAME,
            TRUCK_FLAG = EXCLUDED.TRUCK_FLAG,
            RAIL_FLAG = EXCLUDED.RAIL_FLAG,
            PO_TRANSMISSION_TYPE = EXCLUDED.PO_TRANSMISSION_TYPE,
            PO_TRANSMISSION_VALUE = EXCLUDED.PO_TRANSMISSION_VALUE,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY

            RETURNING CS_VENDOR_ID'''
    )


        # Fetch the generated C1 value
        generated_c1 = cur.fetchone()[0]

        # If C1 was generated or updated, insert/update other tables
        if generated_c1:
            print('insert if')
            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_ORDER_POINT (
            CS_VENDOR_ID,
            MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO,
            USE_CYCLE_STOCK_AS_MIN,
            USE_FAC_WHSE_SS,
            LEAD_TIME_OVRD_DAYS_1,
            LEAD_TIME_OVRD_DAYS_2,
            LEAD_TIME_OVRD_DAYS_3,
            LEAD_TIME_OVRD_DAYS_4,
            LEAD_TIME_OVRD_DAYS_5,
            LEAD_TIME_OVRD_DAYS_6,
            LEAD_TIME_OVRD_DAYS_7,
            APPLY_RATIONING,
            OP_DAYS_1ST_RUN_1,
            OP_DAYS_1ST_RUN_2,
            OP_DAYS_1ST_RUN_3,
            OP_DAYS_1ST_RUN_4,
            OP_DAYS_1ST_RUN_5,
            OP_DAYS_1ST_RUN_6,
            OP_DAYS_1ST_RUN_7,
            OP_DAYS_2ND_RUN_1,
            OP_DAYS_2ND_RUN_2,
            OP_DAYS_2ND_RUN_3,
            OP_DAYS_2ND_RUN_4,
            OP_DAYS_2ND_RUN_5,
            OP_DAYS_2ND_RUN_6,
            OP_DAYS_2ND_RUN_7,
            SHIP_WITH_VENDOR,
            CONNECTING_VENDOR,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            '{row['MAIN_FACILITY']}',
            '{row['BUILD_TO_DAYS_SUPPLY']}',
            '{row['DAYS_TO_BUILD_TO']}',
            '{row['USE_CYCLE_STOCK_AS_MIN']}',
            '{row['USE_FAC_WHSE_SS']}',
            '{row['LEAD_TIME_OVRD_DAYS_1']}',
            '{row['LEAD_TIME_OVRD_DAYS_2']}',
            '{row['LEAD_TIME_OVRD_DAYS_3']}',
            '{row['LEAD_TIME_OVRD_DAYS_4']}',
            '{row['LEAD_TIME_OVRD_DAYS_5']}',
            '{row['LEAD_TIME_OVRD_DAYS_6']}',
            '{row['LEAD_TIME_OVRD_DAYS_7']}',
            '{row['APPLY_RATIONING']}',
            '{row['OP_DAYS_1ST_RUN_1']}',
            '{row['OP_DAYS_1ST_RUN_2']}',
            '{row['OP_DAYS_1ST_RUN_3']}',
            '{row['OP_DAYS_1ST_RUN_4']}',
            '{row['OP_DAYS_1ST_RUN_5']}',
            '{row['OP_DAYS_1ST_RUN_6']}',
            '{row['OP_DAYS_1ST_RUN_7']}',
            '{row['OP_DAYS_2ND_RUN_1']}',
            '{row['OP_DAYS_2ND_RUN_2']}',
            '{row['OP_DAYS_2ND_RUN_3']}',
            '{row['OP_DAYS_2ND_RUN_4']}',
            '{row['OP_DAYS_2ND_RUN_5']}',
            '{row['OP_DAYS_2ND_RUN_6']}',
            '{row['OP_DAYS_2ND_RUN_7']}',
            '{row['SHIP_WITH_VENDOR']}',
            '{row['CONNECTING_VENDOR']}',
            '{row['request_id']}',
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            MAIN_FACILITY = EXCLUDED.MAIN_FACILITY,
            BUILD_TO_DAYS_SUPPLY = EXCLUDED.BUILD_TO_DAYS_SUPPLY,
            DAYS_TO_BUILD_TO = EXCLUDED.DAYS_TO_BUILD_TO,
            USE_CYCLE_STOCK_AS_MIN = EXCLUDED.USE_CYCLE_STOCK_AS_MIN,
            USE_FAC_WHSE_SS = EXCLUDED.USE_FAC_WHSE_SS,
            LEAD_TIME_OVRD_DAYS_1 = EXCLUDED.LEAD_TIME_OVRD_DAYS_1,
            LEAD_TIME_OVRD_DAYS_2 = EXCLUDED.LEAD_TIME_OVRD_DAYS_2,
            LEAD_TIME_OVRD_DAYS_3 = EXCLUDED.LEAD_TIME_OVRD_DAYS_3,
            LEAD_TIME_OVRD_DAYS_4 = EXCLUDED.LEAD_TIME_OVRD_DAYS_4,
            LEAD_TIME_OVRD_DAYS_5 = EXCLUDED.LEAD_TIME_OVRD_DAYS_5,
            LEAD_TIME_OVRD_DAYS_6 = EXCLUDED.LEAD_TIME_OVRD_DAYS_6,
            LEAD_TIME_OVRD_DAYS_7 = EXCLUDED.LEAD_TIME_OVRD_DAYS_7,
            APPLY_RATIONING = EXCLUDED.APPLY_RATIONING,
            OP_DAYS_1ST_RUN_1 = EXCLUDED.OP_DAYS_1ST_RUN_1,
            OP_DAYS_1ST_RUN_2 = EXCLUDED.OP_DAYS_1ST_RUN_2,
            OP_DAYS_1ST_RUN_3 = EXCLUDED.OP_DAYS_1ST_RUN_3,
            OP_DAYS_1ST_RUN_4 = EXCLUDED.OP_DAYS_1ST_RUN_4,
            OP_DAYS_1ST_RUN_5 = EXCLUDED.OP_DAYS_1ST_RUN_5,
            OP_DAYS_1ST_RUN_6 = EXCLUDED.OP_DAYS_1ST_RUN_6,
            OP_DAYS_1ST_RUN_7 = EXCLUDED.OP_DAYS_1ST_RUN_7,
            OP_DAYS_2ND_RUN_1 = EXCLUDED.OP_DAYS_2ND_RUN_1,
            OP_DAYS_2ND_RUN_2 = EXCLUDED.OP_DAYS_2ND_RUN_2,
            OP_DAYS_2ND_RUN_3 = EXCLUDED.OP_DAYS_2ND_RUN_3,
            OP_DAYS_2ND_RUN_4 = EXCLUDED.OP_DAYS_2ND_RUN_4,
            OP_DAYS_2ND_RUN_5 = EXCLUDED.OP_DAYS_2ND_RUN_5,
            OP_DAYS_2ND_RUN_6 = EXCLUDED.OP_DAYS_2ND_RUN_6,
            OP_DAYS_2ND_RUN_7 = EXCLUDED.OP_DAYS_2ND_RUN_7,
            SHIP_WITH_VENDOR = EXCLUDED.SHIP_WITH_VENDOR,
            CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        '''
    )


            cur.execute(
                f'''
                INSERT INTO XXPO_VENDOR_ORDER_INFO (
                    CS_VENDOR_ID,
                    VENDOR_NUMBER,
                    CONNECTING_VENDOR,
                    TRANSIT_DAYS,
                    LAG_DAYS,
                    LEAD_TIME_STATED_TYPE,
                    LEAD_TIME_STATED_DAYS,
                    MINIMUM_BKT_0,
                    MINIMUM_BKT_0_TYPE,
                    MAXIMUM_BKT,
                    MAXIMUM_BKT_TYPE,
                    MIN_BKT_2ND,
                    MIN_BKT_TYPE_2ND,
                    MAX_BKT_2ND,
                    MAX_BKT_TYPE_2ND,
                    FIXED_REVIEW_MON,
                    FIXED_REVIEW_TUE,
                    FIXED_REVIEW_WED,
                    FIXED_REVIEW_THU,
                    FIXED_REVIEW_FRI,
                    FIXED_REVIEW_SAT,
                    FIXED_REVIEW_SUN,
                    VENDOR_SERVICE_LEVEL,
                    DAYS_TO_BUY_OVERRIDE,
                    ORDER_SEQUENCE,
                    LOAD_BLDG_LIMIT_FLG,
                    ORDER_FILTER_FLAG,
                    REQUEST_ID,
                    CREATION_DATE,
                    LAST_UPDATE_DATE,
                    CREATED_BY,
                    LAST_UPDATED_BY
                )
                VALUES (
                    {generated_c1},
                    '{row['VENDOR_NUMBER']}'::numeric,
                    '{row['CONNECTING_VENDOR']}',
                    '{row['TRANSIT_DAYS']}',
                    '{row['LAG_DAYS']}',
                    '{row['LEAD_TIME_STATED_TYPE']}',
                    '{row['LEAD_TIME_STATED_DAYS']}',
                    NULLIF('{row['MINIMUM_BKT_0']}', '')::numeric,
                    '{row['MINIMUM_BKT_0_TYPE']}',
                    NULLIF('{row['MAXIMUM_BKT']}', '')::numeric,
                    '{row['MAXIMUM_BKT_TYPE']}',
                    NULLIF('{row['MIN_BKT_2ND']}', '')::numeric,
                    '{row['MIN_BKT_TYPE_2ND']}',
                    NULLIF('{row['MAX_BKT_2ND']}', '')::numeric,
                    '{row['MAX_BKT_TYPE_2ND']}',
                    '{row['FIXED_REVIEW_MON']}',
                    '{row['FIXED_REVIEW_TUE']}',
                    '{row['FIXED_REVIEW_WED']}',
                    '{row['FIXED_REVIEW_THU']}',
                    '{row['FIXED_REVIEW_FRI']}',
                    '{row['FIXED_REVIEW_SAT']}',
                    '{row['FIXED_REVIEW_SUN']}',
                    NULLIF('{row['VENDOR_SERVICE_LEVEL']}', '')::numeric,
                    '{row['DAYS_TO_BUY_OVERRIDE']}',
                    '{row['ORDER_SEQUENCE']}',
                    '{row['LOAD_BLDG_LIMIT_FLG']}',
                    '{row['ORDER_FILTER_FLAG']}',
                    '{row['request_id']}',
                    current_timestamp,
                    current_timestamp,
                    '-1',
                    '-1'
                )
                ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
                    VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
                    CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
                    TRANSIT_DAYS = EXCLUDED.TRANSIT_DAYS,
                    LAG_DAYS = EXCLUDED.LAG_DAYS,
                    LEAD_TIME_STATED_TYPE = EXCLUDED.LEAD_TIME_STATED_TYPE,
                    LEAD_TIME_STATED_DAYS = EXCLUDED.LEAD_TIME_STATED_DAYS,
                    MINIMUM_BKT_0 = EXCLUDED.MINIMUM_BKT_0,
                    MINIMUM_BKT_0_TYPE = EXCLUDED.MINIMUM_BKT_0_TYPE,
                    MAXIMUM_BKT = EXCLUDED.MAXIMUM_BKT,
                    MAXIMUM_BKT_TYPE = EXCLUDED.MAXIMUM_BKT_TYPE,
                    MIN_BKT_2ND = EXCLUDED.MIN_BKT_2ND,
                    MIN_BKT_TYPE_2ND = EXCLUDED.MIN_BKT_TYPE_2ND,
                    MAX_BKT_2ND = EXCLUDED.MAX_BKT_2ND,
                    MAX_BKT_TYPE_2ND = EXCLUDED.MAX_BKT_TYPE_2ND,
                    FIXED_REVIEW_MON = EXCLUDED.FIXED_REVIEW_MON,
                    FIXED_REVIEW_TUE = EXCLUDED.FIXED_REVIEW_TUE,
                    FIXED_REVIEW_WED = EXCLUDED.FIXED_REVIEW_WED,
                    FIXED_REVIEW_THU = EXCLUDED.FIXED_REVIEW_THU,
                    FIXED_REVIEW_FRI = EXCLUDED.FIXED_REVIEW_FRI,
                    FIXED_REVIEW_SAT = EXCLUDED.FIXED_REVIEW_SAT,
                    FIXED_REVIEW_SUN = EXCLUDED.FIXED_REVIEW_SUN,
                    VENDOR_SERVICE_LEVEL = EXCLUDED.VENDOR_SERVICE_LEVEL,
                    DAYS_TO_BUY_OVERRIDE = EXCLUDED.DAYS_TO_BUY_OVERRIDE,
                    ORDER_SEQUENCE = EXCLUDED.ORDER_SEQUENCE,
                    LOAD_BLDG_LIMIT_FLG = EXCLUDED.LOAD_BLDG_LIMIT_FLG,
                    ORDER_FILTER_FLAG = EXCLUDED.ORDER_FILTER_FLAG,
                    REQUEST_ID = EXCLUDED.REQUEST_ID,
                    CREATION_DATE = EXCLUDED.CREATION_DATE,
                    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                    CREATED_BY = EXCLUDED.CREATED_BY,
                    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                '''
            )



            cur.execute(
        f'''
        INSERT INTO XXPO_VENDOR_COSTING_INFO (
            CS_VENDOR_ID,
            FREIGHT_ALLOW_BUY_RATE,
            FREIGHT_ALLOW_BUY_RATE_TYPE,
            FREIGHT_ALLOW_BUY_RATE_CHANGED,

            PREPAYADD_BUY_RATE_TYPE,
            PREPAYADD_BUY_RATE_CHANGED,
            FREIGHT_BILL_BUY_RATE,
            FREIGHT_BILL_BUY_RATE_TYPE,
            FREIGHT_BILL_BUY_RATE_CHANGED,
            BACKHAUL_BUY_RATE,
            BACKHAUL_BUY_RATE_TYPE,
            BACKHAUL_BUY_RATE_CHANGED,
            PRE_PAY_FLAG,
            PRE_PAY_ADD_FLAG,
            FREIGHT_BILL_FLAG,
            BACKHAUL_FLAG,
            EXTERNAL_BACKHAUL_FLAG,
            SSA_RATE,
            SSA_RATE_TYPE,
            SSA_RATE_BASIS_NET_FLAG,
            SSA_RATE_BASIS_GROSS_FLAG,
            SSA_RATE_BASIS_ACC,
            CASH_DISC_TERMS,
            CASH_DISC_TERMS_DAYS,
            CASH_DISC_TERMS_NET,
            REQUEST_ID,
            CREATION_DATE,
            LAST_UPDATE_DATE,
            CREATED_BY,
            LAST_UPDATED_BY
        )
        VALUES (
            {generated_c1},
            NULLIF('{row['FREIGHT_ALLOW_BUY_RATE']}', '')::numeric,
            '{row['FREIGHT_ALLOW_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['FREIGHT_ALLOW_BUY_RATE_CHANGED']}', '')::date" if row['FREIGHT_ALLOW_BUY_RATE_CHANGED'] else 'NULL'},

            '{row['PREPAYADD_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['PREPAYADD_BUY_RATE_CHANGED']}', '')::date" if row['PREPAYADD_BUY_RATE_CHANGED'] else 'NULL'},
            NULLIF('{row['FREIGHT_BILL_BUY_RATE']}', '')::numeric,
            '{row['FREIGHT_BILL_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['FREIGHT_BILL_BUY_RATE_CHANGED']}', '')::date" if row['FREIGHT_BILL_BUY_RATE_CHANGED'] else 'NULL'},
            NULLIF('{row['BACKHAUL_BUY_RATE']}', '')::numeric,
            '{row['BACKHAUL_BUY_RATE_TYPE']}',
            {f"NULLIF('{row['BACKHAUL_BUY_RATE_CHANGED']}', '')::date" if row['BACKHAUL_BUY_RATE_CHANGED'] else 'NULL'},
            '{row['PRE_PAY_FLAG']}',
            '{row['PRE_PAY_ADD_FLAG']}',
            '{row['FREIGHT_BILL_FLAG']}',
            '{row['BACKHAUL_FLAG']}',
            '{row['EXTERNAL_BACKHAUL_FLAG']}',
            NULLIF('{row['SSA_RATE']}', '')::numeric,
            '{row['SSA_RATE_TYPE']}',
            '{row['SSA_RATE_BASIS_NET_FLAG']}',
            '{row['SSA_RATE_BASIS_GROSS_FLAG']}',
            '{row['SSA_RATE_BASIS_ACC']}',
            '{row['CASH_DISC_TERMS']}',
            '{row['CASH_DISC_TERMS_DAYS']}',
            '{row['CASH_DISC_TERMS_NET']}',
            '{row['request_id']}',
            current_timestamp,
            current_timestamp,
            '-1',
            '-1'
        )
        ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
            FREIGHT_ALLOW_BUY_RATE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE,
            FREIGHT_ALLOW_BUY_RATE_TYPE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE_TYPE,
            FREIGHT_ALLOW_BUY_RATE_CHANGED = EXCLUDED.FREIGHT_ALLOW_BUY_RATE_CHANGED,
            PREPAYADD_BUY_RATE_TYPE = EXCLUDED.PREPAYADD_BUY_RATE_TYPE,
            PREPAYADD_BUY_RATE_CHANGED = EXCLUDED.PREPAYADD_BUY_RATE_CHANGED,
            FREIGHT_BILL_BUY_RATE = EXCLUDED.FREIGHT_BILL_BUY_RATE,
            FREIGHT_BILL_BUY_RATE_TYPE = EXCLUDED.FREIGHT_BILL_BUY_RATE_TYPE,
            FREIGHT_BILL_BUY_RATE_CHANGED = EXCLUDED.FREIGHT_BILL_BUY_RATE_CHANGED,
            BACKHAUL_BUY_RATE = EXCLUDED.BACKHAUL_BUY_RATE,
            BACKHAUL_BUY_RATE_TYPE = EXCLUDED.BACKHAUL_BUY_RATE_TYPE,
            BACKHAUL_BUY_RATE_CHANGED = EXCLUDED.BACKHAUL_BUY_RATE_CHANGED,
            PRE_PAY_FLAG = EXCLUDED.PRE_PAY_FLAG,
            PRE_PAY_ADD_FLAG = EXCLUDED.PRE_PAY_ADD_FLAG,
            FREIGHT_BILL_FLAG = EXCLUDED.FREIGHT_BILL_FLAG,
            BACKHAUL_FLAG = EXCLUDED.BACKHAUL_FLAG,
            EXTERNAL_BACKHAUL_FLAG = EXCLUDED.EXTERNAL_BACKHAUL_FLAG,
            SSA_RATE = EXCLUDED.SSA_RATE,
            SSA_RATE_TYPE = EXCLUDED.SSA_RATE_TYPE,
            SSA_RATE_BASIS_NET_FLAG = EXCLUDED.SSA_RATE_BASIS_NET_FLAG,
            SSA_RATE_BASIS_GROSS_FLAG = EXCLUDED.SSA_RATE_BASIS_GROSS_FLAG,
            SSA_RATE_BASIS_ACC = EXCLUDED.SSA_RATE_BASIS_ACC,
            CASH_DISC_TERMS = EXCLUDED.CASH_DISC_TERMS,
            CASH_DISC_TERMS_DAYS = EXCLUDED.CASH_DISC_TERMS_DAYS,
            CASH_DISC_TERMS_NET = EXCLUDED.CASH_DISC_TERMS_NET,
            REQUEST_ID = EXCLUDED.REQUEST_ID,
            CREATION_DATE = EXCLUDED.CREATION_DATE,
            LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
            CREATED_BY = EXCLUDED.CREATED_BY,
            LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
        ''')


            cur.execute(f'''INSERT INTO XXPO_VENDOR_EDI_INFO (CS_VENDOR_ID,
                VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID,
                CREATION_DATE,
                LAST_UPDATE_DATE,
                CREATED_BY,
                LAST_UPDATED_BY
                ) 
                VALUES (
                {generated_c1},
                '{row['VENDOR_CONTINUITY_REPLEN_FLG']}',
                '{row['request_id']}',
                current_timestamp,
                current_timestamp,
                '-1',
                '-1') 
                ON CONFLICT (CS_VENDOR_ID) 
                DO UPDATE SET VENDOR_CONTINUITY_REPLEN_FLG = EXCLUDED.VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID = EXCLUDED.REQUEST_ID,
                CREATION_DATE = EXCLUDED.CREATION_DATE,
                LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                CREATED_BY = EXCLUDED.CREATED_BY,
                LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                ''')
                
# print(f'Inserted into all Vendor master and all 4 child tables')

# cur.execute(
#             """UPDATE  XXPO_ORCL2CSPOMS_VEND_STG
#                SET PROCESS_STATUS = 'P'
#                where PROCESS_STATUS ='N'
#             """
#         )
